In [ ]:
root_dir = '/Users/mark/Documents/Terminal evaluation report'

fn = '7.5799_2024_te_unep_spccm_msp_asia_pacific_ EE Lighting in Pakistan.pdf'

import os, pathlib

fp = pathlib.Path(root_dir).joinpath(fn)
fp

!pip install pymupdf

import fitz, pymupdf
from pprint import pprint


doc = fitz.open(fp)
doc

TABLE_TITLE_COLOR = 40930
TABLE_TITLE_COLOR_OFFSET = .05
table_title_color_range = (TABLE_TITLE_COLOR * (1 - TABLE_TITLE_COLOR_OFFSET), TABLE_TITLE_COLOR * (1 + TABLE_TITLE_COLOR_OFFSET))

def parse_page(page: fitz.Page):

    show_image(page, 'page')

    tables = page.find_tables()  # locate tables on page
    for table in tables:
        display(table.to_pandas())
    
    text_blocks = page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)["blocks"]
    for block in text_blocks:
        for line in block["lines"]:
            for span in line["spans"]:
                text = span["text"]
                color = span["color"]
                if abs(1 - color / TABLE_TITLE_COLOR) < TABLE_TITLE_COLOR_OFFSET: 
                    pprint(block)

    # page.fin

parse_page(doc[72])

import pandas as pd

def ding():
    global start_page, end_page, table_data
    if len(table_data):
        data = table_data[-1]
        if 'criterion' in [i.strip().lower() for i in data.columns]:
            print(f">> Table<page=({start_page}-{end_page})>")
            display(data)    

table_data = []
header = None
start_page = 0
end_page = 0
for page in doc:
    # print(page)
    for table in page.find_tables():
        # display(table.to_pandas())
        rows = table.to_pandas()
        table_header = table.header.names

        if 'criterion' in [i.strip().lower() for i in table_header]:
            for cell in table.cells:
                # 获取单元格的边界框
                rect = cell.rect
                
                # 获取该区域内的绘图对象
                paths = page.get_drawings(rect)
                
                for path in paths:
                    # 获取线条属性
                    stroke_width = path['width']  # 线条粗细
                    stroke_color = path['color']  # 线条颜色（RGB格式）
                    stroke_opacity = path['opacity']  # 透明度
                    fill_color = path['fill']  # 填充颜色
                    
                    # 获取线条类型
                    is_dashed = path.get('dashes', None)  # 是否是虚线
                    
                    print(f"单元格 ({cell.row}, {cell.col})（） 的属性:")
                    print(f"- 线条粗细: {stroke_width}")
                    print(f"- 线条颜色: {stroke_color}")
                    print(f"- 透明度: {stroke_opacity}")
                    print(f"- 填充颜色: {fill_color}")
                    
            # print({"header": header, "table_header": table_header, "start_page": start_page, "end_page": end_page})
            if table_header != header:
                ding()
                table_data.append(rows)
                start_page = page.number
                end_page = page.number
            else:
                table_data[-1] = pd.concat([table_data[-1], rows], axis=0)
                # print("merged: ")
                # display(table_data[-1])
            header = table_header
            end_page = page.number

ding()

## utils

In [68]:
"""
Utility function for showing images.

Intended to be imported in Jupyter notebooks to display pixmap images.

Invocation: "show_image(item, title)", where item is a PyMuPDF object
which has a "get_pixmap" method, and title is an optional string.

The function executes "item.get_pixmap(dpi=150)" and show the resulting
image.


Dependencies
------------
numpy, matplotlib, pymupdf
"""


def show_image(item, title=""):
    """Display a pixmap.

    Just to display Pixmap image of "item" - ignore the man behind the curtain.

    Args:
        item: any PyMuPDF object having a "get_pixmap" method.
        title: a string to be used as image title

    Generates an RGB Pixmap from item using a constant DPI and using matplotlib
    to show it inline of the notebook.
    """
    DPI = 320  # use this resolution
    import numpy as np
    import matplotlib.pyplot as plt

    # %matplotlib inline
    pix = item.get_pixmap(dpi=DPI)
    img = np.ndarray([pix.h, pix.w, 3], dtype=np.uint8, buffer=pix.samples_mv)
    plt.figure(dpi=DPI)  # set the figure's DPI
    plt.title(title)  # set title of image
    _ = plt.imshow(img, extent=(0, pix.w * 72 / DPI, pix.h * 72 / DPI, 0))